In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
alljobssoups = []
alljoburls = []
alljoburlstems = []

#Determine how many pages of ads are on https://jobs.chronicle.com/jobs
#When collected, was 665, now 852
#For up-to-minute, use RSS feed

for i in range(1, 665):
    #print(i)
    page=requests.get('https://jobs.chronicle.com/jobs/'+str(i)+'/')
    soup = BeautifulSoup(page.text, 'html.parser')
    alljobssoups.append(soup)\
    for a in soup.find_all('a', attrs={'class': 'js-clickable-area-link'}):
        alljoburls.append('https://jobs.chronicle.com'+str(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", "")))
        alljoburlstems.append(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", ""))
    print(str(i)+' is done. '+str(664-i)+' are left.')
    time.sleep(0.5)

In [ ]:
alldescsoups = []
alldesctitle = []
employ = []
loc = []
salary = []
date = []
categories = []
level = []
employtype = []
text = []

In [ ]:
#Check length of alljoburls list for range
for i in range(0,13267):
    page = requests.get(alljoburls[i])
    soup = BeautifulSoup(page.text, 'html.parser')
    alldescsoups.append(soup)
    #number of 'dd' tags determines number of job info fields. 7 is max available fields
    if len(soup.find_all('dd')) == 7:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        employtype.append(soup.find_all('dd')[6].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #'dd' == 6 is when the "level" field is missing, which is usually for admin positions
    elif len(soup.find_all('dd')) == 6:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append('Not Applicable')
        employtype.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Any other cases are treated as an error
    else:
        employ.append('error')
        loc.append('error')
        salary.append('error')
        date.append('error')
        categories.append('error')
        level.append('error')
        employtype.append('error')
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Progress statement
    print(str(i+1)+' is done.'+str(13267-i+1)+' remain.')
    #Delayed to not overly tax server
    time.sleep(0.5)

In [ ]:
#Build one DataFrame from all lists. Will throw an error here if something is wrong.
alljobdat = pd.DataFrame({'title': alldesctitle, 'employer': employ, 'location': loc, 'salary': salary, 'post_date': date, 'categories': categories, 'pos_type': level, 'employ_type': employtype, 'description': text, 'url': alljoburls})
#Export CSV with data
alljobdat.to_csv('chronicle_jobs_utf8.csv', index=False, encoding='utf-8')

In [2]:
#Read in data collected Nov 15 2020
dat = pd.read_csv(r"H:\Dropbox\Kent\FA20\Doc Review\data files\chronicle_jobs_utf8.csv")

In [3]:
dat.keys()

Index(['title', 'employer', 'location', 'salary', 'post_date', 'categories',
       'pos_type', 'employ_type', 'description', 'url'],
      dtype='object')

In [4]:
dat.count()

title          13267
employer       13267
location       13267
salary         13267
post_date      13267
categories     13267
pos_type       13267
employ_type    13267
description    13267
url            13267
dtype: int64

In [5]:
dat['post_date'][0]

'Oct 30, 2020'

In [6]:
dat['post_date'][-1:]

13266    Aug 13, 2019
Name: post_date, dtype: object

In [7]:
dat['categories'].head()

0    Faculty Positions; Science, Technology & Mathe...
1    Faculty Positions; Science, Technology & Mathe...
2    Faculty Positions; Education; Education Admini...
3    Faculty Positions; Health & Medicine; Medicine...
4    Faculty Positions; Arts; Performing Arts; Jobs...
Name: categories, dtype: object

In [8]:
dat['tag_list'] = [x.split(";") for x in dat['categories']]

In [9]:
for i in range(13267-1):
    dat['tag_list'][i] = [x.lstrip() for x in  dat['tag_list'][i]]

In [10]:
all_tags = []

In [11]:
for i in dat['tag_list']:
    for x in i:
        if x not in all_tags:
            all_tags.append(x)

In [12]:
all_tags

['Faculty Positions',
 'Science, Technology & Mathematics',
 'Engineering',
 'Education',
 'Education Administration & Leadership',
 'Health & Medicine',
 'Medicine',
 'Other Health & Medicine',
 'Arts',
 'Performing Arts',
 'Jobs Outside Academe',
 'Non-profit & Government Organizations',
 'Administrative',
 'Business & Administrative Affairs',
 'Chief Business Officers & Vice Presidents',
 'Other Business & Administrative Affairs',
 'Academic Affairs',
 'Other Academic Affairs',
 'Alumni Affairs',
 'Development & Advancement',
 'Social & Behavioral Sciences',
 'Psychology',
 'Deans',
 'Public Relations & Marketing (Campus)',
 'Auxiliary Services',
 'Librarians & Library Administration',
 'Technology Administration/Other',
 'Financial Affairs',
 'Counsellor Education',
 'Curriculum & Instruction',
 'Nursing',
 'Humanities',
 'History',
 'Political Science & International Relations',
 'Other Social & Behavioral Sciences',
 'Professional Fields',
 'Vocational & Technical Fields',
 'Stud

In [13]:
len(all_tags)

172

In [14]:
all_tags[162:172]

[' Business & Administrative Affairs',
 ' Chief Business Officers & Vice Presidents',
 ' Financial Affairs',
 ' Student Affairs',
 ' Financial Aid',
 ' Executive',
 ' Executive Directors',
 ' Faculty Positions',
 ' Business & Management',
 ' Accounting & Finance']

In [15]:
all_tags[162] = 'Business & Administrative Affairs'
all_tags[163] = 'Chief Business Officers & Vice Presidents'
all_tags[164] =  'Financial Affairs'
all_tags[165] =  'Student Affairs'
all_tags[166] =  'Financial Aid'
all_tags[167] =  'Executive'
all_tags[168] =  'Executive Directors'
all_tags[169] =  'Faculty Positions'
all_tags[170] =  'Business & Management'
all_tags[171] =  'Accounting & Finance'
all_tags[162:172]

['Business & Administrative Affairs',
 'Chief Business Officers & Vice Presidents',
 'Financial Affairs',
 'Student Affairs',
 'Financial Aid',
 'Executive',
 'Executive Directors',
 'Faculty Positions',
 'Business & Management',
 'Accounting & Finance']

In [16]:
irr_tags = ['Engineering',
 'Medicine',
 'Business & Administrative Affairs',
 'Chief Business Officers & Vice Presidents',
 'Other Business & Administrative Affairs',
 'Alumni Affairs',
 'Development & Advancement',
 'Deans',
 'Auxiliary Services',
 'Financial Affairs',
 'Counsellor Education',
 'Nursing',
 'History',
 'Vocational & Technical Fields',
 'Student Affairs',
 'Athletics',
 'Kinesiology, Exercise Physiology & Physical Education',
 'Biology & Life Sciences',
 'Surgery',
 'Economics',
 'Business & Administrative Support',
 'Executive',
 'Executive Directors',
 'Mathematics',
 'Chief Academic Officers & Vice Presidents',
 'Music',
 'Academic Advising & Academic Support Services',
 'Chief Student Affairs Officers & Vice Presidents',
 'Management',
 'Special Education',
 'Other Student Affairs',
 'Pediatrics',
 'Other Executive',
 'Health Services',
 'Financial Aid',
 'Facilities Management & Physical Plant',
 'Disabled Student Services',
 'Student Activities & Services',
 'Minority & Multicultural Affairs',
 'Accounting & Finance',
 'Business Administration',
 'Classics',
 'Faculty Affairs',
 'Security Studies',
 'Veterinary Sciences',
 'Agriculture & Animal Sciences',
 'Legal Affairs',
 'Design & Graphic Arts',
 'Conference & Special Event Administration',
 'Criminal Justice & Criminology',
 'Extension Services',
 'Residence Life',
 'Dentistry',
 'Physical Therapy',
 'Chancellors & Presidents',
 'Anthropology & Archaeology',
 'Physics & Space Sciences',
 'Chemistry & Biochemistry',
 'Nutrition',
 'Biotechnology & Bioengineering',
 'Art History',
 'Pharmacology',
 'Architecture',
 'Geology & Earth Sciences',
 'Occupational Therapy',
 'Speech & Hearing Sciences',
 'Food Services Management',
 'Provosts',
 'Educational Service Providers',
 'Geography',
 'Primary Care',
 'Principals & Headmasters',
 'Religious Affairs & Ministry',
 'Superintendents & Commissioners']

In [17]:
len(irr_tags)

74

In [18]:
rel_tags = [x for x in all_tags if x not in irr_tags]
rel_tags = list(set(rel_tags))
len(rel_tags)

88

In [19]:
dat['tag_list'][0]

['Faculty Positions', 'Science, Technology & Mathematics', 'Engineering']

In [20]:
dat.iloc[0]

title          Assistant Professor, Tenure Track, in Quantum ...
employer                                University of Washington
location                               Washington, United States
salary                                      Salary Not specified
post_date                                           Oct 30, 2020
categories     Faculty Positions; Science, Technology & Mathe...
pos_type                                   Tenured/Tenured Track
employ_type                                            Full Time
description    [<div class="block fix-text job-description">\...
url            https://jobs.chronicle.com/job/297334/assistan...
tag_list       [Faculty Positions, Science, Technology & Math...
Name: 0, dtype: object

In [21]:
reldat = pd.DataFrame(columns = dat.keys())
reldat

,title,employer,location,salary,post_date,categories,pos_type,employ_type,description,url,tag_list


In [22]:
bool(set(dat['tag_list'][0]) & set(irr_tags))

True

In [23]:
dat['tag_list'][12]

['Administrative', 'Academic Affairs', 'Librarians & Library Administration']

In [24]:
bool(set(dat['tag_list'][12]) & set(irr_tags))

False

In [25]:
for i in range(13267-1):
    if(set(dat['tag_list'][i]) & set(irr_tags)):
        pass
    else:
        reldat = reldat.append(dat.iloc[i], ignore_index = True)

In [26]:
reldat.head()

,title,employer,location,salary,post_date,categories,pos_type,employ_type,description,url,tag_list
0,"ASSOCIATE VICE PRESIDENT, TEACHING AND LEARNING",Kwantlen Polytechnic University,Canada,Competitive Salary,"Oct 30, 2020",Faculty Positions; Education; Education Admini...,Executive,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297332/associat...,"[Faculty Positions, Education, Education Admin..."
1,Assistant Vice President of Faculty and Creati...,New Jersey Performing Arts Center,"New Jersey, United States",Salary Not specified,"Oct 30, 2020",Faculty Positions; Arts; Performing Arts; Jobs...,Executive,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297330/assistan...,"[Faculty Positions, Arts, Performing Arts, Job..."
2,Associate or Full Professor / Chair,The University of Texas at Tyler,"Texas, United States",Salary Commensurate with experience,"Oct 30, 2020",Faculty Positions; Social & Behavioral Science...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297325/associat...,"[Faculty Positions, Social & Behavioral Scienc..."
3,Special Collections Librarian,Fisk University,"Tennessee, United States",Competitive Salary,"Oct 30, 2020",Administrative; Academic Affairs; Librarians &...,Administrative,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297321/special-...,"[Administrative, Academic Affairs, Librarians ..."
4,Assistant Professor - MSW Program,Carlow University,"Pennsylvania, United States",Competitive Salary,"Oct 30, 2020",Faculty Positions; Social & Behavioral Science...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297315/assistan...,"[Faculty Positions, Social & Behavioral Scienc..."


In [27]:
sum(1 for x in reldat['tag_list'] if 'Faculty Positions' in x)

2335

In [28]:
is_fac = [1 if 'Faculty Positions' in x else 0 for x in reldat['tag_list']]
reldat['is_fac'] = is_fac

In [29]:
is_jour = [1 if 'Journalism' in x else 0 for x in reldat['tag_list']]
reldat['is_jour'] = is_jour

In [30]:
is_socbeh = [1 if ('Social & Behavioral Sciences' or 'Other Social & Behavioral Sciences') in x else 0 for x in reldat['tag_list']]
reldat['is_socbeh'] = is_socbeh

In [31]:
fac_socbeh_list = reldat.loc[(reldat['is_fac'] == 1) & (reldat['is_jour'] == 1)]
fac_socbeh_list

,title,employer,location,salary,post_date,categories,pos_type,employ_type,description,url,tag_list,is_fac,is_jour,is_socbeh
95,Visiting Scholar - Communication - (FAC001592),University of Houston,"Texas, United States",Salary Not Specified,"Oct 28, 2020",Faculty Positions; Communications; Digital Med...,Not Applicable,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/295881/visiting...,"[Faculty Positions, Communications, Digital Me...",1,1,0
225,Assistant Professor - Journalism - Digital Rep...,University of Oklahoma,"Oklahoma, United States",Salary Not specified,"Oct 27, 2020",Faculty Positions; Professional Fields; Journa...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/295437/assistan...,"[Faculty Positions, Professional Fields, Journ...",1,1,0
320,"Adjunct, Introduction to Journalism",Monmouth University,"New Jersey, United States",Salary Not specified,"Oct 26, 2020",Faculty Positions; Professional Fields; Journa...,Adjunct,Part Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/295133/adjunct-...,"[Faculty Positions, Professional Fields, Journ...",1,1,0
590,"2021-22 Open-Rank, Tenure-Track Faculty Positi...",University of Texas at Austin,"Texas, United States",Salary Not specified,"Oct 20, 2020",Faculty Positions; Professional Fields; Journa...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/293791/2021-22-...,"[Faculty Positions, Professional Fields, Journ...",1,1,0
603,2020-21-Non-Tenure Track Faculty Pool - Journa...,University of Texas at Austin,"Texas, United States",Salary Not specified,"Oct 20, 2020",Faculty Positions; Professional Fields; Journa...,Non-Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/293756/2020-21-...,"[Faculty Positions, Professional Fields, Journ...",1,1,0
951,Instructor of Journalism,Ball State University,"Indiana, United States",Salary Not Specified,"Oct 12, 2020",Faculty Positions; Communications; Digital Med...,Not Applicable,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/292130/instruct...,"[Faculty Positions, Communications, Digital Me...",1,1,0
1007,Associate Professor(s) / Assistant Professor(s),The Chinese University of Hong Kong,Hong Kong,Competitive Salary,"Oct 12, 2020",Faculty Positions; Communications; Other Commu...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/291723/associat...,"[Faculty Positions, Communications, Other Comm...",1,1,0
1008,Associate Professor(s) / Assistant Professor(s),The Chinese University of Hong Kong,Hong Kong,Competitive Salary,"Oct 12, 2020",Faculty Positions; Communications; Other Commu...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/291722/associat...,"[Faculty Positions, Communications, Other Comm...",1,1,0
1009,Professor(s) / Associate Professor(s),The Chinese University of Hong Kong,Hong Kong,Competitive Salary,"Oct 12, 2020",Faculty Positions; Communications; Other Commu...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/291721/professo...,"[Faculty Positions, Communications, Other Comm...",1,1,0
1010,Associate Professor(s) / Assistant Professor(s),The Chinese University of Hong Kong,Hong Kong,Competitive Salary,"Oct 12, 2020",Faculty Positions; Communications; Other Commu...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/291720/associat...,"[Faculty Positions, Communications, Other Comm...",1,1,0
